In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from __future__ import print_function

import numpy as np
import tqdm 

from collections import Counter
from glove import Glove, Corpus

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, FeatureAgglomeration
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import TruncatedSVD
from sklearn import metrics
from sklearn.metrics import accuracy_score
np.random.seed(1337)  # for reproducibility

from sklearn.metrics import adjusted_mutual_info_score as AMI
from sklearn.metrics import v_measure_score as VM
from sklearn.metrics.cluster import adjusted_rand_score as ARI

In [1]:
from data import loader_snli, loader_msr

## Utils

In [4]:
def purity_score(y_true, y_pred):
    """Purity score
        Args:
            y_true(np.ndarray): n*1 matrix Ground truth labels
            y_pred(np.ndarray): n*1 matrix Predicted clusters

        Returns:
            float: Purity score
    """
    # matrix which will hold the majority-voted labels
    y_voted_labels = np.zeros(y_true.shape)
    # Ordering labels
    ## Labels might be missing e.g with set like 0,2 where 1 is missing
    ## First find the unique labels, then map the labels to an ordered set
    ## 0,2 should become 0,1
    labels = np.unique(y_true)
    ordered_labels = np.arange(labels.shape[0])
    for k in range(labels.shape[0]):
        y_true[y_true==labels[k]] = ordered_labels[k]
    # Update unique labels
    labels = np.unique(y_true)
    # We set the number of bins to be n_classes+2 so that 
    # we count the actual occurence of classes between two consecutive bins
    # the bigger being excluded [bin_i, bin_i+1[
    bins = np.concatenate((labels, [np.max(labels)+1]), axis=0)

    for cluster in np.unique(y_pred):
        hist, _ = np.histogram(y_true[y_pred==cluster], bins=bins)
        # Find the most present label in the cluster
        winner = np.argmax(hist)
        y_voted_labels[y_pred==cluster] = winner

    return accuracy_score(y_true, y_voted_labels)

### Load Data

In [5]:
PATH = 'msr-paraphrase-corpus/'
# PATH = 'snli_1.0/'
data = loader_msr(PATH)

In [6]:
print(len(data['train-x']), max(data['train-y']))
print(len(data['test-x']), max(data['test-y']))

5344 2590
2274 1126


# Baseline

## Approx number of clusters

In [161]:
def choose_n_cluster(part):
    counter = Counter(data[part + '-y'])
    n_clusters = int(len(data[part + '-x']) / np.mean(list(counter.values())))
#     n_clusters = int(len(data[part + '-x']) / np.median(list(counter.values())))
    return n_clusters

In [162]:
choose_n_cluster('train')

2591

In [163]:
choose_n_cluster('test')

1127

## Vectorizing Data

In [11]:
vectorizer = TfidfVectorizer(ngram_range=(1, 4), stop_words='english')

In [12]:
vectorizer.fit(data['train-x'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 4), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [13]:
data['train-vec'] = vectorizer.transform(data['train-x'])
data['test-vec'] = vectorizer.transform(data['test-x'])

In [14]:
data['train-vec'].shape, data['test-vec'].shape

((5344, 123108), (2274, 123108))

## PCA decomposition

In [15]:
Xpca = TruncatedSVD(300)

In [16]:
Xpca.fit(data['train-vec'])

TruncatedSVD(algorithm='randomized', n_components=300, n_iter=5,
       random_state=None, tol=0.0)

In [17]:
data['train-pca'] = Xpca.transform(data['train-vec'])
data['test-pca'] = Xpca.transform(data['test-vec'])

# Clustering

In [164]:
part = 'test'
n_clusters = choose_n_cluster(part)
n_clusters

1127

In [165]:
grouper = KMeans(n_clusters=n_clusters)

In [ ]:
label_predict = grouper.fit_predict(data[part + '-pca'])

In [167]:
label_true = data[part + '-y']

In [51]:
label_predict[:10]

array([1652, 1652, 1666, 1666,  125, 2535, 1692, 1692,  320,  320])

## Score

### Random preds

#### ARI

In [43]:
random_aris = []
for i in range(100):
    random_preds = np.random.randint(0, max(label_true) + 1, size=len(label_true))
    ari = ARI(label_true, random_preds)
    random_aris.append(ari)

In [44]:
np.mean(random_aris), np.median(random_aris)

(5.64277064333405e-05, -1.457397025335591e-05)

#### Purity scores

In [44]:
random_purity_scores = []
for i in range(100):
    random_preds = np.random.randint(0, max(label_true) + 1, size=len(label_true))
    purity_score_ = purity_score(label_true, random_preds)
    random_purity_scores.append(purity_score_)

In [45]:
(np.mean(random_purity_scores), np.median(random_purity_scores))

(0.09359653388743977, 0.09354967063088554)

### Clustering score

#### ARI

In [52]:
ARI(label_true, label_predict)

0.7640486551055635

#### Purity scores

In [53]:
purity_score(label_true, label_predict)__

0.9418038922155688

# GloVe

In [17]:
part = 'test'

In [129]:
glove = Glove().load_stanford('glove.6B.300d.txt')

In [35]:
gw2v = lambda word: glove.word_vectors[glove.dictionary[word]]

### Make vectors

In [18]:
vectors = []
for sentence in tqdm.tqdm(data[part + '-x']):
#     print (test_sentence)
    sentence_vectors = []
    for word in sentence.split():
        #print (word)
        try:
            wv = gw2v(word.lower())
            sentence_vectors.append(wv)
        except KeyError:
            pass
    vectors.append(np.mean(sentence_vectors, axis=0))

100%|██████████| 2274/2274 [00:00<00:00, 16226.07it/s]


In [19]:
vectors = np.array(vectors)

In [20]:
vectors.shape

(2274, 300)

### Kmeans

In [21]:
n_clusters = choose_n_cluster(part)

In [22]:
grouper = KMeans(n_clusters=n_clusters)

In [23]:
grouper.fit(vectors, n_clusters)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=1127, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [24]:
label_predict = grouper.predict(vectors)
label_true = data[part + '-y']

In [25]:
ARI(label_true, label_predict)

0.7391433525668684

In [26]:
purity_score(label_true, label_predict)

0.9120492524186455

# Algomerative clustering

In [27]:
n_clusters = choose_n_cluster(part)
algomerative_grouper = AgglomerativeClustering(n_clusters=n_clusters)

In [28]:
label_predict = algomerative_grouper.fit_predict(vectors, n_clusters)
label_true = data[part + '-y']

In [29]:
label_predict

array([312, 312, 235, ..., 865, 987, 987])

In [30]:
ARI(label_true, label_predict)

0.8419865125918745

In [31]:
purity_score(label_true, label_predict)

0.9423922603342129

## Train Glove

In [57]:
import re

In [78]:
def build_tokenizer(token_pattern):
    token_pattern = re.compile(token_pattern)
    return lambda doc: token_pattern.findall(doc.lower())

In [79]:
bulider = build_tokenizer('(?u)\\b\\w\\w+\\b')

In [130]:
corpus = Corpus()
corpus.fit(map(bulider, data['train-x']))

In [131]:
%%time 
glove.fit(corpus.matrix, epochs=3000, no_threads=4)
glove.add_dictionary(corpus.dictionary)

CPU times: user 2h 21min 19s, sys: 2.67 s, total: 2h 21min 21s
Wall time: 35min 55s


In [132]:
gw2v = lambda word: glove.word_vectors[glove.dictionary[word]]

In [147]:
part = 'train'
vectors = []
for sentence in tqdm.tqdm(data[part + '-x']):
#     print (test_sentence)
    sentence_vectors = []
    for word in sentence.split():
#         print(word)
        try:
            wv = gw2v(word.lower())
            sentence_vectors.append(wv)
        except KeyError:
            pass
        except IndexError:
            pass
    vectors.append(np.mean(sentence_vectors, axis=0))

100%|██████████| 5344/5344 [00:00<00:00, 16695.18it/s]


In [148]:
vectors = np.array(vectors)

In [149]:
vectors.shape

(5344, 300)

### Kmeans

In [150]:
n_clusters = choose_n_cluster(part)

In [151]:
grouper = KMeans(n_clusters=n_clusters)

In [152]:
grouper.fit(vectors, n_clusters)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=2591, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [153]:
label_predict = grouper.predict(vectors)
label_true = data[part + '-y']

In [154]:
ARI(label_true, label_predict)

0.4271054004040746

In [155]:
purity_score(label_true, label_predict)

0.7900449101796407

# Algomerative clustering

In [156]:
n_clusters = choose_n_cluster(part)
algomerative_grouper = AgglomerativeClustering(n_clusters=n_clusters)

In [157]:
label_predict = algomerative_grouper.fit_predict(vectors, n_clusters)
label_true = data[part + '-y']

In [158]:
label_predict

array([1169, 1169, 1024, ...,   26,  989, 2541])

In [159]:
ARI(label_true, label_predict)

0.5928495377256653

In [160]:
purity_score(label_true, label_predict)

0.8280314371257484

# Выводы

### Векторизация фраз

Для векторизации фраз сначала использовался известный алгоритм `Tf-Idf`. Он далает неплохую векторизацию, взвешивая слова и выдяляя наиболее важные, но к сожалению он не учитвыет синонимы или связь между словами. Эти проблемы решает предобученный `Glove`. При использовании этого типа векторизации и находя средний вектор предложения, любой алгоритм класторизации показал неплохие результаты.

Дообучение `Glove` не принесло прироста в качестве, а наоборот ухудшило его. Это может быть связано с некоторыми фактами. Например с нехваткой ресурсов: хорошие модели векторизации, такие как `ELMO` или `BERT`, обучались на новейших процессорах Intel и использовали огромное количество памяти (>8 Gb RAM).

### Кластеризация векторов

Для обучения использовались два алгоритма: `KMeans` и `Алгомитиративная кластеризация`. Оба из них просты в использовании.

Но оба из них плохи тем, что мы заранее должны знать кол-во кластеров, на которое их нужно разбивать, а так же они оба чувствутительны к выбросам. Такой проблемы нет в `DBSCAN`, но в нашей задачи из-за большого кол-ва объектов и большого значение variance, этот алгоритм не эффективен. Проблему кол-ва кластеров можно решить оценкой распределения количества размеров кластеров. С выбросами можно бороться на этапе векторизации (`Glove` решает эту проблему)

Использование `KMeans` эффективно по памяти, так как это итеративный алгоритм. `Агломиративная кластеризация` наоборот использует большое кол-во памяти, так как находит попарные расстояния между объектами. Данные устроены так, что имеется большое кол-во кластеров (много групп парафраз с двумя предожениями). Поэтому на этих данных эффективнее всего пользоваться `Агломиративной кластеризацией`. 

### Результаты


<table style="border-style: solid; border-collapse: collapse;">
  <tr>
    <th rowspan="2" style="border: 1px solid gray;     text-align: center;">Data</th>
    <th rowspan="2" style="border: 1px solid gray;     text-align: center;">Method Vectorization</th>
    <th rowspan="2" style="border: 1px solid gray;     text-align: center;">Method Clustering</th>
    <th colspan="2" style="border: 1px solid gray;     text-align: center;">ARI</th>
    <th colspan="2" style="border: 1px solid gray;     text-align: center;">Purity Score</th>
  </tr>
  <tr>
    <td style="border: 1px solid gray;     text-align: center;">train</td>
    <td style="border: 1px solid gray;     text-align: center;">test</td>
    <td style="border: 1px solid gray;     text-align: center;">train</td>
    <td style="border: 1px solid gray;     text-align: center;">test</td>
  </tr>
  <tr>
    <td rowspan="6" style="border: 1px solid gray;     text-align: center;">Microsoft Research Paraphrase Corpus</td>
    <td style="border: 1px solid gray;     text-align: center;">Tf-Idf + PCA</td>
    <td style="border: 1px solid gray;     text-align: center;">KMeans</td>
    <td style="border: 1px solid gray">0,304009</td>
    <td style="border: 1px solid gray">0,149801</td>
    <td style="border: 1px solid gray">0,86</td>
    <td style="border: 1px solid gray">0,75</td>
  </tr>
  <tr>
    <td style="border: 1px solid gray;     text-align: center;">Tf-Idf + PCA</td>
    <td style="border: 1px solid gray;     text-align: center;">Agglomerative</td>
    <td style="border: 1px solid gray">0,764049</td>
    <td style="border: 1px solid gray">0,413312</td>
    <td style="border: 1px solid gray">0,94</td>
    <td style="border: 1px solid gray">0,85</td>
  </tr>
  <tr>
    <td style="border: 1px solid gray;     text-align: center;">Glove</td>
    <td style="border: 1px solid gray;     text-align: center;">KMeans</td>
    <td style="border: 1px solid gray">0,733050</td>
    <td style="border: 1px solid gray">0,751013</td>
    <td style="border: 1px solid gray">0,92</td>
    <td style="border: 1px solid gray">0,92</td>
  </tr>
  <tr>
    <td style="border: 1px solid gray;     text-align: center;">Glove</td>
    <td style="border: 1px solid gray;     text-align: center;">Agglomerative</td>
    <th style="border: 1px solid gray">0,845933</th>
    <th style="border: 1px solid gray">0,848819</th>
    <th style="border: 1px solid gray">0,95</th>
    <th style="border: 1px solid gray">0,95</th>
  </tr>
  <tr>
    <td style="border: 1px solid gray;     text-align: center;">Glove + fit</td>
    <td style="border: 1px solid gray;     text-align: center;">Agglomerative</td>
    <td style="border: 1px solid gray">0,592850</td>
    <td style="border: 1px solid gray">0,597578</td>
    <td style="border: 1px solid gray">0,83</td>
    <td style="border: 1px solid gray">0,83</td>
  </tr>
  <tr>
    <td style="border: 1px solid gray;     text-align: center;">Random</td>
    <td style="border: 1px solid gray;     text-align: center;">Random</td>
    <td style="border: 1px solid gray">-0,000012</td>
    <td style="border: 1px solid gray">0,000058</td>
    <td style="border: 1px solid gray">0,42</td>
    <td style="border: 1px solid gray">0,43</td>
  </tr>
  <tr>
    <td rowspan="4" style="border: 1px solid gray;     text-align: center;">The Stanford Natural Language Inference (SNLI) Corpus</td>
    <td style="border: 1px solid gray;     text-align: center;">Tf-Idf + PCA</td>
    <td style="border: 1px solid gray;     text-align: center;">KMeans</td>
    <td style="border: 1px solid gray">0,022221</td>
    <td style="border: 1px solid gray">0,078349</td>
    <td style="border: 1px solid gray">0,22</td>
    <td style="border: 1px solid gray">0,51</td>
  </tr>
  <tr>
    <td style="border: 1px solid gray;     text-align: center;">Glove</td>
    <td style="border: 1px solid gray;     text-align: center;">KMeans</td>
    <td style="border: 1px solid gray">0,031683</td>
    <td style="border: 1px solid gray">0,158066</td>
    <td style="border: 1px solid gray">0,24</td>
    <td style="border: 1px solid gray">0,56</td>
  </tr>
  <tr>
    <td style="border: 1px solid gray;     text-align: center;">Glove</td>
    <td style="border: 1px solid gray;     text-align: center;">Agglomerative</td>
    <td style="border: 1px solid gray">--</td>
    <th style="border: 1px solid gray">0,215346</th>
    <td style="border: 1px solid gray">--</td>
    <th style="border: 1px solid gray">0,58</th>
  </tr>
  <tr>
    <td style="border: 1px solid gray;     text-align: center;">Random</td>
    <td style="border: 1px solid gray;     text-align: center;">Random</td>
    <td style="border: 1px solid gray">-0,000002</td>
    <td style="border: 1px solid gray">0,000015</td>
    <td style="border: 1px solid gray">0,09</td>
    <td style="border: 1px solid gray">0,31</td>
  </tr>
</table>

Анализ алгоритмов кластеризации и векторизации данных полностью соответсвуют ожиданиям